In [2]:
import requests
import pandas as pd

# stat variables
season_id = '2022-23'
per_mode = 'PerGame'
season_post = 'Playoffs'

# stats url with season_id and per_mode concatenated within
player_info_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
player_info_url_2='https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2022-23&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

response = requests.get(url=player_info_url, headers=headers).json()
response_2 = requests.get(url=player_info_url_2, headers=headers).json()

player_info = response['resultSets'][0]['rowSet']
player_info_2 = response_2['resultSets'][0]['rowSet']

columns_list = [
    "PLAYER_ID",
    "PLAYER_NAME",
    "NICKNAME",
    "TEAM_ID",
    "TEAM_ABBREVIATION",
    "AGE",
    "GP",
    "W",
    "L",
    "W_PCT",
    "MIN",
    "FGM",
    "FGA",
    "FG_PCT",
    "FG3M",
    "FG3A",
    "FG3_PCT",
    "FTM",
    "FTA",
    "FT_PCT",
    "OREB",
    "DREB",
    "REB",
    "AST",
    "TOV",
    "STL",
    "BLK",
    "BLKA",
    "PF",
    "PFD",
    "PTS",
    "PLUS_MINUS",
    "NBA_FANTASY_PTS",
    "DD2",
    "TD3",
    "WNBA_FANTASY_PTS",
    "GP_RANK",
    "W_RANK",
    "L_RANK",
    "W_PCT_RANK",
    "MIN_RANK",
    "FGM_RANK",
    "FGA_RANK",
    "FG_PCT_RANK",
    "FG3M_RANK",
    "FG3A_RANK",
    "FG3_PCT_RANK",
    "FTM_RANK",
    "FTA_RANK",
    "FT_PCT_RANK",
    "OREB_RANK",
    "DREB_RANK",
    "REB_RANK",
    "AST_RANK",
    "TOV_RANK",
    "STL_RANK",
    "BLK_RANK",
    "BLKA_RANK",
    "PF_RANK",
    "PFD_RANK",
    "PTS_RANK",
    "PLUS_MINUS_RANK",
    "NBA_FANTASY_PTS_RANK",
    "DD2_RANK",
    "TD3_RANK",
    "WNBA_FANTASY_PTS_RANK",
]

In [34]:

"""
IDEA: get the names of all teams top scorers (players that scored over 20 per game during regluar season or playoffs) 
and see if there is a trend between teams whose star players underperformed and players who elevated their scoring in the playoffs
and whether or not the team advanced through the first round of the playoffs
"""

# points comparison
playoffs_df = pd.DataFrame(player_info, columns=columns_list)
regular_df = pd.DataFrame(player_info_2, columns=columns_list)
#nba_df.to_csv('nba_playoff_season_stats_2023.csv', index=False)
# players_df = nba_df[['PLAYER_NAME', 'PTS']]
# display(players_df.head())

#display(regular_df)
#display(playoffs_df)

# simplify DataFrames ofr data that we need for this analysis
playoffs_df = playoffs_df[['PLAYER_NAME', 'PTS', 'TEAM_ABBREVIATION']]
regular_df = regular_df[['PLAYER_NAME', 'PTS', 'TEAM_ABBREVIATION']]

# get list of player names from reg season and playoffs
playoff_players = playoffs_df['PLAYER_NAME'].tolist()
regular_players = regular_df['PLAYER_NAME'].tolist()

# players that have stats listed in both playoffs and regular season
names = []
for i in range(len(playoff_players)):
    for j in range(len(regular_players)):
        if playoff_players[i] == regular_players[j]:
            names.append(playoff_players[i])

# list to insert player point differentials as a tuples
stats = []
#dic = {}
top_players_dic = {}
better_than_reg = []
worse_than_reg = []

# loop through names of player that are in the playoffs
for i in range(len(names)):
    reg_stat = regular_df[regular_df['PLAYER_NAME']==names[i]]['PTS'].values[0]
    playoff_stat = playoffs_df[playoffs_df['PLAYER_NAME']==names[i]]['PTS'].values[0]
    
    #dic[names[i]] = reg_stat, playoff_stat
    if reg_stat > 25 or playoff_stat > 25:
        team = playoffs_df[playoffs_df['PLAYER_NAME']==names[i]]['TEAM_ABBREVIATION'].values[0]
        stat = (names[i],reg_stat, playoff_stat, team)
        stats.append(stat)
        #top_players_dic[names[i]] = reg_stat, playoff_stat
        
        if reg_stat > playoff_stat:
            worse_than_reg.append(stat)
        else:
            better_than_reg.append(stat)


#display(len(top_players_dic))
#display("worse", worse_than_reg)
#display("better", better_than_reg)
final_df = pd.DataFrame(stats, columns=['NAME','RS_PTS','P_PTS', 'TEAM'])
#display(final_df)

first_round_exits = ['MIN', 'ATL', 'MIL', 'MEM', 'BKN', 'CLE', 'LAC', 'SAC']
advanced = ['PHX', 'LAL', 'MIA', 'DEN', 'GSW', 'PHI', 'BOS', 'NYK']

reg_better_df = pd.DataFrame(worse_than_reg, columns=['NAME','RS_PTS','P_PTS', 'TEAM'])
playoff_better_df = pd.DataFrame(better_than_reg, columns=['NAME','RS_PTS','P_PTS', 'TEAM'])

print("these players underperformed as scorers")
display(reg_better_df)

print("these players elevated their scoring")
display(playoff_better_df)

# after looking at the data, it appears that teams with top scorers that underperformed in the scoring category actually had more success in the first round of the playoffs
# this could be due to the importance of role players and depth/defense in the playoffs

these players underperformed as scorers


,NAME,RS_PTS,P_PTS,TEAM
0,Anthony Davis,25.9,22.6,LAL
1,Donovan Mitchell,28.3,23.2,CLE
2,Giannis Antetokounmpo,31.1,23.3,MIL
3,Ja Morant,26.2,24.6,MEM
4,Jaylen Brown,26.6,22.7,BOS
5,Jayson Tatum,30.1,27.7,BOS
6,Joel Embiid,33.1,23.7,PHI
7,Julius Randle,25.1,16.6,NYK
8,Kevin Durant,29.1,29.0,PHX
9,LeBron James,28.9,24.5,LAL


these players elevated their scoring


,NAME,RS_PTS,P_PTS,TEAM
0,Anthony Edwards,24.6,31.6,MIN
1,De'Aaron Fox,25.0,27.4,SAC
2,Devin Booker,27.8,33.7,PHX
3,Jalen Brunson,24.0,27.8,NYK
4,Jamal Murray,20.0,27.7,DEN
5,Jimmy Butler,22.9,28.8,MIA
6,Kawhi Leonard,23.8,34.5,LAC
7,Nikola Jokic,24.5,29.9,DEN
8,Stephen Curry,29.4,30.5,GSW
9,Trae Young,26.2,29.2,ATL
